# Linear Programming: Letting Computers make our decisions
The purpose of this notebook is to show a practical example on how to solve a LP problem. In this case, we will be solving a sports scheduling problem with linear programming

## Data Preparation

In [1]:
# We import the libraries we will use
import os 
from ortools.linear_solver import pywraplp
import pandas as pd

In [2]:
# We load the datasets we will use as inputs to the model
df_dist_matrix = pd.read_csv(os.getcwd() + '/Code Inputs/distance_matrix.csv')
df_positions = pd.read_csv(os.getcwd() + '/Code Inputs/positions.csv')

We create a dictionary position_dict that relates teams with its position

In [3]:
def get_position_dict(df_positions):
    # We create a dictionary that relates teams with its position
    position_dict = {}
    for index, row in df_positions.iterrows():
        position_dict[row['Team']] = row['Position']
    return position_dict

We create a dictionary dist_matrix that will serve as a cost matrix

In [4]:
def get_dist_matrix(df_dist_matrix, N):
    # We create a dictionary dist_matrix that will serve as a cost matrix. Example
    # dist_matrix[(i, j)] = distance between teams i and j
    dist_matrix = {}
    for index, row in df_dist_matrix.iterrows():
        team_one = row['Team']
        for team_two in N:
            dist_matrix[(team_one, team_two)] = row[team_two]
    return dist_matrix


We create a dictionary called int_dict that holds the interest generated by the match between teams i and j at matchday t

In [5]:
def get_interest_matrix(positions, N, m):
    # We create a dictionary called int_dict
    # int_dict[(i, j, t)] = interest of match betweens teams i and j at matchday t
    int_dict = {}
    
    for team_one in N:
        pos_team_one = positions[team_one]
        for team_two in N:
            pos_team_two = positions[team_two]
            for t in range(m):
                interest = t / (1 + abs(pos_team_one - pos_team_two))
                int_dict[(team_one, team_two, t)] = interest
    
    return int_dict

In [6]:
# We create a class that has all the data and methods used in the model
class SportsSchedule:
    def __init__(self, df_dist_matrix, df_positions):
        """
        Class that has all the data and methods used in the model
        :param df_dist_matrix: dataframe that represents a distance matrix between all teams
        :param df_position: dataframe that has information of the position of each team in last year's tournament
        """
        # We setup the inputs as variables
        self.df_dist_matrix = df_dist_matrix
        self.df_positions = df_positions
        
        # We setup the model parameters
        self.N = list(self.df_dist_matrix['Team'])
        self.n = len(self.N)
        self.m, self.r = 2*self.n - 2, self.n - 1
        
        # We create a dictionary that relates teams with its position
        self.positions = get_position_dict(self.df_positions)
            
        # We create a dictionary dist_matrix that will serve as a cost matrix
        self.dist_matrix = get_dist_matrix(self.df_dist_matrix, self.N)
                
        # We create a dictionary called int_dict which holds the interest of match betweens teams i and j at matchday t
        self.int_dict = get_interest_matrix(self.positions, self.N, self.m)

## Model setup

### Model and Variable creation
We will setup the solver (i.e. we will create the model) and declare the decision variables of the model

### Match variables
$$x_{ijt}$$ A binary variable that is equal to 1 if team i plays at home against team j at matchday t

In [7]:
# We create the dictionary x_var_dict
# x_var_dict[(i,j,t)] will have the solver variable for the binary variable that will be equal to one if team i 
# hosts team j in matchday t

def match_variable_creation(Schedule, solver):
    # We check that the variable hasn't been already created
    created_xvariables = []
    x_var_dict = {}
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for t in range(Schedule.m):
                if str(team_one+team_two+str(t)) not in created_xvariables:
                    x_var_dict[(team_one, team_two, t)] = solver.BoolVar(str('x_'+team_one+team_two+str(t)))
                    created_xvariables.append(str(team_one+team_two+str(t)))
    return x_var_dict, solver

### Travel variables
$$y_{ijkt}$$ A binary variable that is equal to 1 if team i travels from j to k after matchday t

In [8]:
def travel_variable_creation(Schedule, solver):
    # y_var_dict[(i,j,k,t)] will have the solver variable for the binary variable that will be equal to one if team i 
    # travels from j to k after matchday t
    y_var_dict = {}
    created_yvariables = []
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m):
                    if str(team_one+team_two+team_three+str(t)) not in created_yvariables:
                        y_var_dict[(team_one, team_two, team_three, t)] = solver.BoolVar(
                            str('y_'+team_one+team_two+team_three+str(t)))
                        created_yvariables.append(str(team_one+team_two+team_three+str(t)))
    return y_var_dict, solver

In [9]:
def model_and_variable_creation(Schedule):
    """
    A function that creates a or tools solver model and creates the decision variables used
    :param Schedule: A SportsSchedule instance
    return
    solver: an or tools solver model
    x_var_dict = a dictionary with the binary "x" variables
    y_var_dict = a dictionary with the binary "y" variables
    """
    # We create the solver
    solver = pywraplp.Solver('simple_mip_program',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # We create the dictionary x_var_dict that hold the "match" variables
    x_var_dict, solver = match_variable_creation(Schedule, solver)
    
    # We the dictionary y_var_dict that will hold the travel variables
    y_var_dict, solver = travel_variable_creation(Schedule, solver)
    
    return solver, x_var_dict, y_var_dict

### Constraint Creation
We setup the different constraints for the problem

#### Teams shouldn't play themselves
We start by creating the constraint that prohibits teams from playing themselves in a particular matchday. This can be represented by the following equation:
$$ x_{iit} = 0,    \forall  i \in N, \forall t \in T $$

In [10]:
def teams_shouldnt_play_themselves_constraint(Schedule, solver, x_var_dict, y_var_dict):
    for team in Schedule.N:
        for t in range(Schedule.m):
            ct = solver.Constraint(0, 0, 'MatchesBetween'+team+'round'+str(t)+'are forbidden')
            ct.SetCoefficient(x_var_dict[(team, team, t)], 1)
    
    return solver

#### Teams play against one team each matchday
We create a constraint that makes each team play against one rival each matchday. This can be represented by the following equation:

$$ \sum_{j, j \ne i} x_{ijt} + \sum_{j, j \ne i} x_{jit} = 1,    \forall  i \in N, \forall t \in T $$

In [11]:
def one_match_per_matchday(Schedule, solver, x_var_dict, y_var_dict):
    for team_one in Schedule.N:
        for t in range(Schedule.m):
            ct = solver.Constraint(1, 1, 'MatchesTeam'+team_one+'Round'+str(t))
            for team_two in Schedule.N:
                if team_one != team_two:
                    ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
                    ct.SetCoefficient(x_var_dict[(team_two, team_one, t)], 1)
    return solver

#### Teams face once in the first round
We create a constraint that makes every pair of rivals face once in the first round of the tournament. This can be represented by the following equation:
$$ \sum_{t, t \leq r} x_{ijt} + \sum_{t, t \leq r} x_{jit} = 1,  \forall  i, j \in N, i \ne j $$

In [12]:
def one_match_per_rival_first_round(Schedule, solver, x_var_dict, y_var_dict):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            if team_one != team_two:
                ct = solver.Constraint(1, 1, 'FirstRound'+team_one+team_two)
                for t in range(Schedule.r):
                    ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
                    ct.SetCoefficient(x_var_dict[(team_two, team_one, t)], 1)
    return solver

#### Teams face twice in the tournament
We create a constraint that makes every pair of rivals face twice in the tournament. This can be represented by the following equation:
$$ \sum_{t} x_{ijt} + \sum_{t} x_{jit} = 2,  \forall  i, j \in N, i \ne j $$

In [13]:
def two_matches_per_rival_tournament(Schedule, solver, x_var_dict, y_var_dict):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            if team_one != team_two:
                ct = solver.Constraint(2, 2, 'TwoMatchesTournament'+team_one+team_two)
                for t in range(Schedule.m):
                    ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
                    ct.SetCoefficient(x_var_dict[(team_two, team_one, t)], 1)
    return solver

#### Teams can host a rival just once
We create a constraint that makes a team i host another team j just once in the tournament. This constraints exists in order to prevent a situation in which team i hosts both matches against team j. This can be represented by the following equation:

$$ \sum_{t} x_{ijt} = 1,    \forall  i, j \in N, i \ne j $$

In [14]:
def host_rival_once(Schedule, solver, x_var_dict, y_var_dict):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            if team_one != team_two:
                ct = solver.Constraint(1, 1, team_one+'HostsOnce'+team_two)
                for t in range(Schedule.m):
                    ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
    return solver

### Link x variables and y variables
We have to link the match variables ("x") with the trip variables ("y"). E.g

$$ x_{jit} = 1 \land x_{kit+1} = 1    \Rightarrow  y_{ijkt} = 1$$

This means: if team i plays as an away team against j in matchday t, and as an away team against k in matchday t+1 (both "x"s), then team i should travel from j to k after t ("y"). This translates to:

$$ y_{ijkt} \geq x_{jit} + x_{kit+1} - 1,    \forall i, j, k \in N, \forall t \in T, t < m $$
$$ y_{iikt} \geq x_{ijt} + x_{kit+1} - 1,    \forall i, j, k \in N, \forall t \in T, t < m $$
$$ y_{ijit} \geq x_{jit} + x_{ikt+1} - 1,    \forall i, j, k \in N, \forall t \in T, t < m $$
$$ y_{iiit} \geq x_{ijt} + x_{ikt+1} - 1,    \forall i, j, k \in N, \forall t \in T, t < m $$

However, this restrictions will only be created when the objective function tries to minimize total distance.

As an example, we will set up the first link restriction between the match and the travel variables

$$ y_{ijkt} \geq x_{jit} + x_{kit+1} - 1,    \forall i, j, k \in N, \forall t \in T, t < m $$

In [15]:
def link_x_y_away_away(Schedule, solver, x_var_dict, y_var_dict, constraint_names):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m-1):
                    ctname = team_one+'TravelsFrom'+team_two+'To'+team_three+'AfterMatchday'+str(t)
                    if ctname not in constraint_names:
                        ct = solver.Constraint(0, 1, ctname)
                        ct.SetCoefficient(x_var_dict[(team_two, team_one, t)], 1)
                        ct.SetCoefficient(x_var_dict[(team_three, team_one, t+1)], 1)
                        ct.SetCoefficient(y_var_dict[(team_one, team_two, team_three, t)], -1)
                        constraint_names.append(ctname)

    return solver, constraint_names

In [16]:
def link_x_y_home_away(Schedule, solver, x_var_dict, y_var_dict, constraint_names):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m-1):
                    ctname = team_one+'TravelsFrom'+team_one+'To'+team_three+'AfterMatchday'+str(t)
                    if ctname not in constraint_names:
                        ct = solver.Constraint(0, 1, ctname)
                        ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
                        ct.SetCoefficient(x_var_dict[(team_three, team_one, t+1)], 1)
                        ct.SetCoefficient(y_var_dict[(team_one, team_one, team_three, t)], -1)
                        constraint_names.append(ctname)
                        
    return solver, constraint_names

def link_x_y_away_home(Schedule, solver, x_var_dict, y_var_dict, constraint_names):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m-1):
                    ctname = team_one+'TravelsFrom'+team_two+'To'+team_one+'AfterMatchday'+str(t)
                    if ctname not in constraint_names:
                        ct = solver.Constraint(0, 1, ctname)
                        ct.SetCoefficient(x_var_dict[(team_two, team_one, t)], 1)
                        ct.SetCoefficient(x_var_dict[(team_one, team_three, t+1)], 1)
                        ct.SetCoefficient(y_var_dict[(team_one, team_two, team_one, t)], -1)
                        constraint_names.append(ctname)
                        
    return solver, constraint_names

def link_x_y_home_home(Schedule, solver, x_var_dict, y_var_dict, constraint_names):
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m-1):
                    ctname = team_one+'TravelsFrom'+team_one+'To'+team_one+'AfterMatchday'+str(t)
                    if ctname not in constraint_names:
                        ct = solver.Constraint(0, 1, ctname)
                        ct.SetCoefficient(x_var_dict[(team_one, team_two, t)], 1)
                        ct.SetCoefficient(x_var_dict[(team_one, team_three, t+1)], 1)
                        ct.SetCoefficient(y_var_dict[(team_one, team_one, team_one, t)], -1)
                        constraint_names.append(ctname)
                        
    return solver, constraint_names

In [17]:
def link_x_y_variables(Schedule, solver, x_var_dict, y_var_dict, obj_funct):
    if obj_funct == 'min_distance':
        constraint_names = []
        solver, constraint_names = link_x_y_away_away(Schedule, solver, x_var_dict, y_var_dict, constraint_names)
        solver, constraint_names = link_x_y_home_away(Schedule, solver, x_var_dict, y_var_dict, constraint_names)
        solver, constraint_names = link_x_y_away_home(Schedule, solver, x_var_dict, y_var_dict, constraint_names)
        solver, constraint_names = link_x_y_home_home(Schedule, solver, x_var_dict, y_var_dict, constraint_names)
    else:
        pass
    return solver

For the sake of being more neat, we create a function that creates all the constraints

In [18]:
def add_constraints(Schedule, solver, x_var_dict, y_var_dict, obj_funct):
    solver = teams_shouldnt_play_themselves_constraint(Schedule, solver, x_var_dict, y_var_dict)
    solver = one_match_per_matchday(Schedule, solver, x_var_dict, y_var_dict)
    solver = one_match_per_rival_first_round(Schedule, solver, x_var_dict, y_var_dict)
    solver = two_matches_per_rival_tournament(Schedule, solver, x_var_dict, y_var_dict)
    solver = host_rival_once(Schedule, solver, x_var_dict, y_var_dict)
    solver = link_x_y_variables(Schedule, solver, x_var_dict, y_var_dict, obj_funct)
    return solver

### Objective Function Creation
We setup the different objective functions

#### If the objective function is equal to 'min_distance'
Then the solver will try to minimize the total distance. This can be represented by the following equation:

$$ min \sum_{i} \sum_{j} \sum_{k} \sum_{t} \bar{d_{jk}}*y_{ijkt} $$

In [19]:
def min_distance_obj_function(Schedule, solver, x_var_dict, y_var_dict):
    objective = solver.Objective()
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for team_three in Schedule.N:
                for t in range(Schedule.m):
                    objective.SetCoefficient(
                            y_var_dict[(team_one, team_two, team_three, t)], Schedule.dist_matrix[(team_two, team_three)])
    objective.SetMinimization()
    return solver, x_var_dict, y_var_dict, objective

#### If the objective function is equal to 'max_interest'
Then the solver will try to maximize the interest in the last matchdays. This can be represented by the following equation

$$ max \sum_{i} \sum_{j} \sum_{t} x_{ijt}*p_{ijt} $$

In [20]:
def max_interest_obj_function(Schedule, solver, x_var_dict, y_var_dict):
    objective = solver.Objective()
    for team_one in Schedule.N:
        for team_two in Schedule.N:
            for t in range(Schedule.m):
                objective.SetCoefficient(x_var_dict[(team_one, team_two, t)], Schedule.int_dict[(team_one, team_two, t)])
    objective.SetMaximization()
    return solver, x_var_dict, y_var_dict, objective

In [21]:
def define_objective_function(Schedule, solver, x_var_dict, y_var_dict, obj_funct):
    if obj_funct == 'min_distance':
        solver, x_var_dict, y_var_dict, objective = min_distance_obj_function(Schedule, solver, x_var_dict, y_var_dict)
    elif obj_funct == 'max_interest':
        solver, x_var_dict, y_var_dict, objective = max_interest_obj_function(Schedule, solver, x_var_dict, y_var_dict)
    else:
        pass
    
    solver.Solve()
    
    return solver, x_var_dict, y_var_dict, objective

## Solution analysis

### Tranlation of the model's output to a valid schedule

In [22]:
def get_schedule(Schedule, x_var_dict):
    # We create a list of matches for each team
    matches = {}
    for team in Schedule.N:
        matches[team] = [0]*(Schedule.m)
    # We look for each value in the x variable dictionary
    for variable in x_var_dict:
        if round(x_var_dict[variable].solution_value()) == 1:
            home, away, matchday = list(variable)[0], list(variable)[1], list(variable)[2]
            # We fill the dictionary
            matches[home][matchday], matches[away][matchday] = away, str("@"+home)
    # We create the dataframe
    df_schedule = pd.DataFrame()
    df_schedule['Matchday'] = list(range(1, Schedule.m+1))
    for team in matches:
        df_schedule[team] = matches[team]
    return df_schedule

### Extract schedule's KPIs
In order to compare both solutions, we calculate for a given schedule, the total distance and the total interest generated

In [23]:
def total_distance_calculation(Schedule, df_schedule):
    total_distance, teams = 0, Schedule.N
    for i in range(len(df_schedule)):
        # For the distance analysis, we don't consider the first match
        if df_schedule['Matchday'][i] != 1:
            for team in teams:
                # We get the rival for this match and the previous one
                rival_this_match, rival_previous_match = df_schedule[team][i], df_schedule[team][i-1]              
                # We get the home stadium for this match and the previous one, seeing if the first item is "@"
                if rival_this_match[0] == '@':
                    home_this_match = rival_this_match[1:]
                else:
                    home_this_match = rival_this_match
                if rival_previous_match[0] == '@':
                    home_previous_match = rival_previous_match[1:]
                else:
                    home_previous_match = rival_previous_match
                total_distance += Schedule.dist_matrix[(home_this_match, home_previous_match)]
                
    return total_distance

We calculate the total interest of the tournament. Remember the interest of the match between teams i and j in matchday j is calculated as 

$$ p_{ijt} = t/(1+|pos_{i}-pos_{j}|) $$

In [24]:
def total_interest_calculation(Schedule, df_schedule):
    total_interest, teams = 0, Schedule.N
    for i in range(len(df_schedule)):
        # We calculate the interest of each match
        for team in teams:
            rival_this_match = df_schedule[team][i]
            # We consider just the teams that are playing home to avoid duplicating results
            if rival_this_match[0] != '@':
                total_interest += Schedule.int_dict[(team, rival_this_match, i)]
    return total_interest

In [25]:
def schedule_analysis(Schedule, df_schedule):
    total_distance = total_distance_calculation(Schedule, df_schedule)
    total_interest = total_interest_calculation(Schedule, df_schedule)
    return total_distance, total_interest

## Main project pipeline
We summarize the whole project in a few lines of code

In [26]:
for obj_funct in ['min_distance', 'max_interest']:
    
    # Create Schedule instance
    Schedule = SportsSchedule(df_dist_matrix, df_positions)
    
    # Model and variable creation
    solver, x_var_dict, y_var_dict = model_and_variable_creation(Schedule)
    
    # Constraint Creation
    solver = add_constraints(Schedule, solver, x_var_dict, y_var_dict, obj_funct)
    
    # Objective function definition
    solver, x_var_dict, y_var_dict, objective = define_objective_function(Schedule, solver, x_var_dict, y_var_dict, obj_funct)
    
    # Solution analysis
    df_schedule = get_schedule(Schedule, x_var_dict)
    total_distance, total_interest = schedule_analysis(Schedule, df_schedule)
    
    # We print the results
    print("*"*50)
    print(f"Schedule KPIS with the following objective funcion: {obj_funct}")
    print(f"Total distance: {total_distance} km")
    print(f"Total interest generated: {total_interest}")

**************************************************
Schedule KPIS with the following objective funcion: min_distance
Total distance: 15604 km
Total interest generated: 4.934398934398935
**************************************************
Schedule KPIS with the following objective funcion: max_interest
Total distance: 19392 km
Total interest generated: 5.419247419247419
